In [1]:
import pandas as pd
import numpy as np

from skrub import TableVectorizer
import xgboost as xgb
from sklearn.pipeline import Pipeline


In [2]:
# Import the files
df_train = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/train.parquet")
df_test = pd.read_parquet("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/data/final_test.parquet")
external_data_cleaned = pd.read_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/external_data/external_data_cleaned.csv")

In [3]:
# we convert 'date' column to datetime in all datasets
external_data_cleaned['date'] = pd.to_datetime(external_data_cleaned['date'])
df_train['date'] = pd.to_datetime(df_train['date']).astype('datetime64[ns]')
df_test['date'] = pd.to_datetime(df_test['date']).astype('datetime64[ns]')

# Sort datasets by date
external_data_cleaned.sort_values('date', inplace=True)
df_train.sort_values('date', inplace=True)
df_test.sort_values('date', inplace=True)

# we merge the data together  by performing the nearest match in terms of date :
training_set_merged = pd.merge_asof(df_train, external_data_cleaned, on='date', direction='nearest')
testing_set_merged = pd.merge_asof(df_test, external_data_cleaned, on='date', direction='nearest')

In [4]:
# Extract the date feature on different time scales :

def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    # creation of a binary varible depicting if day in weekend
    X["is_weekend"] = np.where(X["weekday"] + 1 > 5, 1, 0)

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

training_set_merged = _encode_dates(training_set_merged)
testing_set_merged = _encode_dates(testing_set_merged)


In [5]:
X_train = training_set_merged.drop(columns=["bike_count", "log_bike_count"])
y_train = training_set_merged["log_bike_count"]

X_test = testing_set_merged.copy()

In [6]:
# Preprocessing pipeline
pipeline = Pipeline(
    steps=[
        ('preprocessor', TableVectorizer()),
        ('model', xgb.XGBRegressor()),  # Note the parentheses to instantiate the model
    ]
)

In [7]:
# Fit Pipeline to Training Data
pipeline.fit(X_train, y_train)

# Make Predictions on Test Data
y_predictions = pipeline.predict(X_test)


In [8]:
print(y_predictions)

[0.40861917 1.5641835  1.7676523  ... 1.8979452  2.9409177  2.2760088 ]


In [ ]:
pd.DataFrame(y_predictions, columns=["log_bike_count"]).reset_index().rename(
    columns={"index": "Id"}
).to_csv("/Users/louisleibovici/Documents/VS_Code/Bike_counters DSB Project/bike_counters/predictions_option_2_vsimple_weather_preprocessed.csv", index=False)